# Tutorial - Analyzing Historical VSTOXX Data
This notebook is based on the [Analyzing Historical VSTOXX Data](http://www.eurexchange.com/advanced-services/vstoxx/01_Analyzing_Historical_VSTOXX_Data.html) tutorial from the [VSTOXX Advanced Services Tutorials](http://www.eurexchange.com/advanced-services/vstoxx/). This notebook shows how you can perform a similar analysis using IBM Knowledge Anyhow Workbench and [SoftLayer Object Storage](http://sldn.softlayer.com/reference/objectstorageapi). The analysis is performed on historical data from the [VSTOXX](http://www.stoxx.com/indices/index_information.html?symbol=V2TX) and the [EURO STOXX 50](http://www.stoxx.com/indices/index_information.html?symbol=sx5e) indices.

This notebook will cover the following steps:

1. Import SoftLayer Object Storage credentials file into your workbench
2. Connect to an object store
3. Extract data from the object store
4. Analyze the extracted data

## Works Cited

* Eurex Exchange, “Analyzing Historical VSTOXX Data” Eurex Advanced Services. Retrieved 28-Oct-2014, from http://www.eurexchange.com/advanced-services/vstoxx/01_Analyzing_Historical_VSTOXX_Data.html.

## Import SoftLayer Object Storage Credentials File

You must provide credentials when connecting to SoftLayer Object Storage.  Credentials have been made available for this tutorial in a [JSON](http://json.org/) file.  **[Click here to import the credentials file](/tutorials/eurex/sl_object_store_credentials.json)** into your workbench.

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You only need to import the credentials file once.  Once you import the credentials file into your workbench, you can re-use the file from other notebooks to connect to the object store.</div>

## Connect to the Object Store

This notebook makes use of the [SoftLayer Object Storage Python Client](https://github.com/softlayer/softlayer-object-storage-python) package to connect to SoftLayer Object Storage.  You must install this package using the `pip` package installer as shown in the following code cell:

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You only need to install this package once.  It will persist in your workbench.</div>

In [ ]:
!pip install softlayer-object-storage

Import the [SoftLayer Object Storage Python Client](https://github.com/softlayer/softlayer-object-storage-python) to interact with SoftLayer Object Storage. Also import the [JSON module](https://docs.python.org/2/library/json.html#) to get the credentials from the provided JSON file. Then create a connection to your object store using the credentials from the JSON file.

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You can re-use the code below in other notebooks to create a connection to the object store.</div>

In [ ]:
import object_storage
import json

with file('/resources/sl_object_store_credentials.json') as f:
    sl_key = json.load(f)
sl_storage = object_storage.get_client(sl_key['username'], sl_key['api_key'], sl_key['server'])

## Extract Data from the Object Store

Your object store contains historical data for the VSTOXX and the EURO STOXX 50 indices. Specifically, the data represents the daily closing values:
* from 01/01/1999 to the last available trading day (when the data was pulled) for the VSTOXX index
* from 12/31/1986 to the last available trading day (when the data was pulled) for the EURO STOXX 50 index

Each data set has been loaded into its own [container](http://sldn.softlayer.com/article/introduction-object-storage#Container) [object](http://sldn.softlayer.com/article/introduction-object-storage#Object). You can use the following Python modules to facilitate the extraction and analysis of the data in your object store:

* [StringIO module](https://docs.python.org/2/library/stringio.html) to read and write files
* [Pandas module](http://pandas.pydata.org/) to facilitate data analysis
* [datetime module](https://docs.python.org/2/library/datetime.html) to filter data

In [ ]:
import StringIO
import pandas
import datetime

Create a DataFrame from the VSTOXX data.

In [ ]:
vs_container_object = sl_storage.get_object("smarterfinancemarketplace","eurex/tutorials/vstoxx/vs.txt")
vs_data = StringIO.StringIO(vs_container_object.read())
vs_df = pandas.DataFrame.from_csv(vs_data, sep=',', index_col=0, parse_dates=True)

Create a DataFrame from the EURO STOXX 50 data.

In [ ]:
es_container_object = sl_storage.get_object("smarterfinancemarketplace","eurex/tutorials/vstoxx/es.txt")
es_data = StringIO.StringIO(es_container_object.read())
es_df = pandas.DataFrame.from_csv(es_data, sep=';', index_col=0, parse_dates=True)

Create a DataFrame of the major indices only (V2TX from the VSTOXX and SX5E from the EURO STOXX 50). Additionally, remove any EURO STOXX 50 data prior to 1999 so that it's consistent with the VSTOXX data.

In [ ]:
st50_df = pandas.DataFrame({'EUROSTOXX' : es_df['SX5E'][es_df.index > datetime.datetime(1999, 1, 1)],
                            'VSTOXX' : vs_df['V2TX'][vs_df.index > datetime.datetime(1999, 1, 1)]})

## Analyze the Extracted Data
Using your data from SoftLayer Object Storage, you can proceed with the analysis workflow outlined in the [Basic Eurex Analysis of Historical Data](http://www.eurexchange.com/advanced-services/vstoxx/01_Analyzing_Historical_VSTOXX_Data.html#index-2).

Configure the matplotlib module to render graphs in the notebook:

In [ ]:
%matplotlib inline

### Plot the daily closing values of the EURO STOXX 50 and VSTOXX indices

In [ ]:
from pylab import *
st50_df.plot(subplots=True, figsize=(9, 6), color='blue', grid=True)

#### Observation
The charts above show the closing values of each index at the given point in time. The two indices seem to be highly negatively correlated. When the EURO STOXX 50 goes up, the VSTOXX comes down and vice versa. This phenomenon is sometimes called the leverage effect (see ["The Leverage Effect Puzzle: Disentangling Sources of Bias
at High Frequency"](https://www.princeton.edu/~yacine/leverage.pdf), ["Is the 'Leverage Effect' a Leverage Effect?"](http://web-docs.stern.nyu.edu/salomon/docs/creditdebtmarkets/S-CDM-00-09.pdf)).

### Plot the daily absolute returns of the EURO STOXX 50 and VSTOXX indices

In [ ]:
h = st50_df.diff().hist(figsize=(10, 5), color='b', alpha=.5, bins=100)

#### Observation
The absolute return value shows the increase or decrease in the closing value of the index. The charts above show the frequencies of absolute return values for each index. The x axes shows the absolute return value. The y axes shows the frequency of each value. For both indices, the absolute return value is close to 0 most of the time. However, the range of absolute return values is greater in the EURO STOXX 50 index. It makes sense that the range would be greater for the EURO STOXX 50 index given its higher value.

### Plot the daily percentage changes of the EURO STOXX 50 and the VSTOXX indices

In [ ]:
st50_df.pct_change().hist(figsize=(10, 5), color='b', alpha=.5, bins=100)

#### Observation
It is also useful to look at the change in value as a percentage. The charts above show the frequencies of percentage changes in closing value for each index. The x axes shows the percentage change in value. The y axes shows the frequency of each percentage change. For both indices, the absolute change in closing value is close to 0 most of the time. However, the range of percentage change is greater in the STOXX index. It makes sense that the range would be greater for the STOXX index given its lower value.

### Plot the daily log returns of the EURO STOXX 50 and the VSTOXX indices

Looking at the absolue returns and percentage changes of the indices provides some insight, but looking at the daily log return values may provide a better perspective (see ["Use of logarithms in economics"](http://econbrowser.com/archives/2014/02/use-of-logarithms-in-economics), ["Why Log Returns"](http://quantivity.wordpress.com/2011/02/21/why-log-returns/), ["Logarithmic or continuously compounded return"](http://en.wikipedia.org/wiki/Rate_of_return#Logarithmic_or_continuously_compounded_return))

In [ ]:
import numpy as np
log_rets = np.log(st50_df / st50_df.shift(1))

In [ ]:
log_rets.plot(subplots=True, figsize=(9, 6), color='blue', grid=True)

#### Observation
The leverage effect seen in the daily closing value charts from above is not apparent in these charts. Calculate the correlation to be sure about the existence of the leverage effect between the EURO STOXX 50 and VSTOXX indices.

You can generate a correlation matrix from the log return values

In [ ]:
log_rets.corr()

or you can compute the correlation directly between the two time series

In [ ]:
log_rets['EUROSTOXX'].corr(log_rets['VSTOXX'])

The log returns of both indices are indeed highly negatively correlated. This is confirmation of the initial observation which implies that risk increases with dropping index levels. If index levels rise then risks come down in general.

## Additional Tutorials
Additional tutorials for the IBM Knowledge Anyhow Workbench are available on our [Welcome](/pages/welcome) page.